In [4]:
# Code for ETL operations on Country-GDP data

# Importing the required libraries
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import sqlite3
from datetime import datetime 

def log_progress(message):
    ''' This function logs the mentioned message of a given stage of the
    code execution to a log file. Function returns nothing'''
    timestamp_format = '%Y-%h-%d-%H:%M:%S' # Year-Monthname-Day-Hour-Minute-Second 
    now = datetime.now() # get current timestamp 
    timestamp = now.strftime(timestamp_format) 
    with open("./code_log.txt","a") as f: 
        f.write(timestamp + ' : ' + message + '\n')    
    
def extract(url, table_attribs):
    ''' This function aims to extract the required
    information from the website and save it to a data frame. The
    function returns the data frame for further processing. '''
    # Send a request to get the page content
    page = requests.get(url).text
    data = BeautifulSoup(page,'html.parser')
    # Find the specific heading "By market capitalization"
    headings = data.find_all(['h1', 'h2', 'h3', 'h4', 'h5', 'h6'])
    #print("Available headings on the page:")
    #for heading in headings:
        #print(heading.text.strip())

    # Find the heading "By market capitalization" (adjust the string based on what's printed)
    heading = data.find(lambda tag: tag.name in ['h2', 'h3', 'h4'] and "capitalization" in tag.text)
    #heading = data.find('h2', string='By market capitalization')
    if heading:
        # Find the next table after the heading
        table = heading.find_next('table')
        if table:
            # Extract table rows
            rows = table.find_all('tr')
            table_data = []
            for row in rows:
                cols = row.find_all(['th', 'td'])
                cols = [col.text.strip() for col in cols]
                table_data.append(cols)

        # Create DataFrame
            df = pd.DataFrame(table_data[1:], columns=table_data[0])
            df = df.iloc[:, 1:3]
            df = df.rename(columns={
                 df.columns[0]: table_attribs[0],
                 df.columns[1]: table_attribs[1]
            })
            #df = df.set_index(table_attribs[0])
            return df
        else:
            print("Table not found after the heading.")
            return None
    else:
        print("Heading 'By market capitalization' not found.")
        return None
    

def transform(df, csv_path1):
    ''' This function accesses the CSV file for exchange rate
    information, and adds three columns to the data frame, each
    containing the transformed version of Market Cap column to
    respective currencies'''
    # Read the CSV file into a DataFrame
    exchange_rate = pd.read_csv(csv_path1)
    
    # Convert the DataFrame into a dictionary with Col_1_header as keys and Col_2_header as values
    #exchange_rate_dict = df.set_index(col_1_header).to_dict()[col_2_header]
    dict = exchange_rate.set_index('Currency').to_dict()['Rate']
    df['MC_USD_Billion'] = pd.to_numeric(df['MC_USD_Billion'], errors='coerce')
    df['MC_GBP_Billion'] = [np.round(x*dict['GBP'],2) for x in df['MC_USD_Billion']]
    df['MC_EUR_Billion'] = [np.round(x*dict['EUR'],2) for x in df['MC_USD_Billion']]
    df['MC_INR_Billion'] = [np.round(x*dict['INR'],2) for x in df['MC_USD_Billion']]

    return df

def load_to_csv(df, output_path):
    ''' This function saves the final data frame as a CSV file in
    the provided path. Function returns nothing.'''

    df.to_csv(output_path, index=False)
 
def load_to_db(df, sql_connection, table_name):
    ''' This function saves the final data frame to a database
    table with the provided name. Function returns nothing.'''
    df.to_sql(table_name, sql_connection, if_exists='replace', index=False)
def run_query(query_statement, sql_connection):
    ''' This function runs the query on the database table and
    prints the output on the terminal. Function returns nothing. '''      
    #Execute the query and load the result into a DataFrame
    df_query_result = pd.read_sql(query_statement, sql_connection)
        
    # Print the query result
    print("Query output:")
    print(df_query_result)
    return df_query_result
''' Here, you define the required entities and call the relevant
functions in the correct order to complete the project. Note that this
portion is not inside any function.'''
url = 'https://en.wikipedia.org/wiki/List_of_largest_banks'
table_attribs = ["Name", "MC_USD_Billion"]
db_name = 'Banks.db'
table_name = 'Largest_banks'
csv_path1 = './exchange_rate.csv'
csv_path = './Largest_banks_data.csv'


log_progress('Preliminaries complete. Initiating ETL process')
df = extract(url, table_attribs)
log_progress('Data extraction complete. Initiating Transformation process')
df = transform(df,csv_path1)

log_progress('Data transformation complete. Initiating loading process')
load_to_csv(df, csv_path)

log_progress('Data saved to CSV file')

sql_connection = sqlite3.connect('Banks.db')
log_progress('SQL Connection initiated.')


load_to_db(df, sql_connection, table_name)
#print(f'The market capitalization of the 5th largest bank in billion EUR is {df["MC_EUR_Billion"][4]}')
#print(df)
log_progress('Data loaded to Database as table. Running the query')
query_statement1 = "SELECT * FROM Largest_banks"
run_query(query_statement1, sql_connection)
query_statement2 = "SELECT AVG(MC_GBP_Billion) FROM Largest_banks"
run_query(query_statement2, sql_connection)
query_statement3 = "SELECT Name from Largest_banks LIMIT 5"
run_query(query_statement3, sql_connection)
log_progress('Process Complete.')
sql_connection.close()

Query output:
                                      Name  MC_USD_Billion  MC_GBP_Billion  \
0                           JPMorgan Chase          551.03          440.82   
1                          Bank of America          288.96          231.17   
2  Industrial and Commercial Bank of China          249.28          199.42   
3                              Wells Fargo          208.41          166.73   
4               Agricultural Bank of China          207.79          166.23   
5                            Bank of China          171.35          137.08   
6                  China Construction Bank          166.19          132.95   
7                                     HSBC          161.05          128.84   
8                           Morgan Stanley          155.58          124.46   
9                                HDFC Bank          153.01          122.41   

   MC_EUR_Billion  MC_INR_Billion  
0          512.46        45707.94  
1          268.73        23969.23  
2          231.83  